In [1]:
import numpy as np
from scipy.stats import binned_statistic
from tqdm import tqdm, trange
import seaborn
import matplotlib.pyplot as plt
import os
import sys


# box = sys.argv[1]
box = 'Box_n50_0_1400'
curr_run_fname = '/oak/stanford/orgs/kipac/aemulus/aemulus_nu/%s/'%(box)
rockstar_dir = curr_run_fname+'output/rockstar/'

f = open(rockstar_dir+'savelist.txt', 'r')
savelist = f.read().split()
f.close()

N_snapshots = len(savelist)

i=0

import pickle

NvMs = {}

In [2]:
NvM_fname = '/oak/stanford/orgs/kipac/users/delon/aemulusnu_massfunction/'+box+'_NvsM.pkl'
NvM_f = open(NvM_fname, 'rb')
NvMs = pickle.load(NvM_f)
NvM_f.close()


In [3]:
jackknife = {}

f_pos = open('/oak/stanford/orgs/kipac/users/delon/aemulusnu_massfunction/'+box+'_pos', 'r')


for a in NvMs:
    snapshot_pos  = f_pos.readline().strip().split(',')
    snapshot_pos  = [np.array(pos.split(), dtype=np.float32) for pos in snapshot_pos if pos != '']
    snapshot_pos  = np.array(snapshot_pos)

    bin_cnters = NvMs[a]['M']
    N = NvMs[a]['N']
    vol = NvMs[a]['vol']
    Mpart = NvMs[a]['Mpart']
    edge_pairs = NvMs[a]['edge_pairs']
    bin_idx = NvMs[a]['bin_idx']
    print(a, np.min(bin_idx), np.max(bin_idx))
    print(len(bin_idx), len(N))
    #redefine the edges that we'll jackknife on 
    edges = [edge[0] for edge in edge_pairs]
    edges += [edge_pairs[-1][1]]    

    #now lets get to spatial jackknife
    N_DIVS = 8 #each axis is diided into N_DIVS parts so in total the box
               #is divided into N_DIVS**3 boxes

    #compute the size of each smaller cube
    ϵ = vol*10**(-6)
    cube_vol = (vol+ε) / N_DIVS**3 #need ϵ to properly handle halos directly on boundary 
    cube_size = np.cbrt(cube_vol)

    #compute the indices of the smaller cube that each point belongs to
    cube_indices = (snapshot_pos / cube_size).astype(int)

    #cube_indices has assignment of halo to 3d position of a voxel
    #ravel_multi_index indexes the voxels in 3D with a single integer
    cube_assignment = np.ravel_multi_index(cube_indices.T, (N_DIVS, N_DIVS, N_DIVS), order='F')
    
    bin_counts = []
    
    print(len(cube_assignment), len(bin_idx))
    for i in trange(N_DIVS**3):
        current_cube = np.where(cube_assignment == i)
        curr_N = np.zeros_like(N)
        for halo in bin_idx[current_cube]:
            #halo=1 corresponds to first bin 
            if(halo==0): #not in any bin 
                continue
            curr_N[halo-1] += 1
        #get the number count of halos in the mass bins in this subcube
        bin_counts += [curr_N]
    bin_counts = np.array(bin_counts)
    mean_counts = np.mean(bin_counts, axis=0)
    dev_counts = bin_counts - mean_counts
#     print('aaaa')
#     print(np.shape(bin_counts))
#     print(np.shape(dev_counts))
#     print(np.shape(mean_counts))
#     print('aaaa')

    cov_counts = np.zeros((len(mean_counts), len(mean_counts)))    

    for i in range(N_DIVS**3):
        # Remove the i-th sub-cube from the sample and calculate the jackknife estimate
        leave_out_idx = np.where(np.arange(N_DIVS**3) != i)
        jackknife_counts = np.mean(bin_counts[leave_out_idx], axis=0)
        dev_jackknife = jackknife_counts - mean_counts
#         print(np.shape(dev_jackknife))
#         print(np.shape(dev_counts[i]))

        cov_counts += np.outer(dev_counts[i], dev_jackknife)

    jackknife_covariance = (N_DIVS**3 - 1)/N_DIVS**3 * cov_counts


#     print(len(N), jackknife_covariance.shape)
    jackknife[a] = [bin_counts, jackknife_covariance]
f_pos.close()


0.25 0 15
2974588 15
2974588 2974588


100%|██████████| 512/512 [00:01<00:00, 392.27it/s]


0.302679 0 15
5155478 15
5155478 5155478


100%|██████████| 512/512 [00:02<00:00, 220.82it/s]


0.366459 0 15
7272025 15
7272025 7272025


...

100%|██████████| 512/512 [00:05<00:00, 94.40it/s]


0.953321 0 15
10321598 15
10321598 10321598


100%|██████████| 512/512 [00:05<00:00, 91.56it/s]


1.0 0 15
10264571 15
10264571 10264571


100%|██████████| 512/512 [00:05<00:00, 96.50it/s]


In [4]:
jackknife_covs_fname = '/oak/stanford/orgs/kipac/users/delon/aemulusnu_massfunction/'+curr_run_fname.split('/')[-2]+'_jackknife_covs.pkl'
jackknife_covs_f = open(jackknife_covs_fname, 'wb')
pickle.dump(jackknife, jackknife_covs_f)
jackknife_covs_f.close()